In [1]:
# ── Cell 1: Initialization & Imports ─────────────────────────────────
# Enable auto-reload so changes to derivatives.py are picked up without restarting the kernel.
%load_ext autoreload
%autoreload 2

# Standard libraries
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

# Import our option classes (with full Greeks) from derivatives.py
from derivatives import Share, EuropeanCall, AmericanPut, BarrierOption, BasketCall

print("✅ Loaded derivatives.py and core libraries.")

tickers = ["BHP.AX", "CBA.AX", "WES.AX", "CSL.AX", "WDS.AX", "MQG.AX"]



✅ Loaded derivatives.py and core libraries.


In [2]:
# ── Cell 2: Spot Prices & Discount Function ─────────────────────────
# Fetch ASX closing prices for our tickers on the trade date
trade_date = "2025-05-16"

# Define a flat-curve discount function (5% p.a.) as placeholder
r        = 0.05
discount = lambda t: np.exp(-r * t)


In [3]:
# ── Cell 3: Realised Volatility Calculation ─────────────────────────
# Download 1 year of historical closes to compute realised vols
hist = yf.download(tickers, end=trade_date, period="1y", progress=False)["Close"]
rets = hist.pct_change().dropna()                # daily returns
vol  = (rets.std() * np.sqrt(252)).to_dict()     # annualised vol σ√252

print("Realised vols (annualised):")
for tkr, σ in vol.items():
    print(f"  {tkr:7s}: {σ:.2%}")


YF.download() has changed argument auto_adjust default to True
Realised vols (annualised):
  BHP.AX : 22.53%
  CBA.AX : 21.39%
  CSL.AX : 19.06%
  MQG.AX : 24.82%
  WDS.AX : 29.25%
  WES.AX : 20.30%


In [4]:
### ── Cell 4: Option Instantiation & Pricing ──────────────────────────


import numpy as np
import pandas as pd

trade_date = "2025-05-16"

# Share objects
bhp_share = Share("BHP.AX", trade_date)
cba_share = Share("CBA.AX", trade_date)
wes_share = Share("WES.AX", trade_date)
csl_share = Share("CSL.AX", trade_date)
wds_share = Share("WDS.AX", trade_date)
mqg_share = Share("MQG.AX", trade_date)

# Helper for year fraction
def year_frac(start, end):
    return (pd.to_datetime(end) - pd.to_datetime(start)).days / 365

# Instantiate options
T1 = year_frac(trade_date, "2027-09-15")
bhp = EuropeanCall(
    Share    = bhp_share,
    K        = 0.98 * bhp_share.get_price(),
    T        = T1,
    discount = discount
)

T2 = year_frac(trade_date, "2026-05-15")
cba = AmericanPut(
    Share    = cba_share,
    K        = 170.0,
    T        = T2,
    discount = discount
)

wes = BarrierOption(
    Share    = wes_share,
    K        = 80.0,
    T        = T1,
    discount = discount,
    barrier  = 100.0
)

T3 = year_frac(trade_date, "2025-07-17")
basket_shares = [bhp_share, csl_share, wds_share, mqg_share]

weights = np.array([0.10, 0.35, 0.15, 0.40])

basket = BasketCall(
    share_list = basket_shares,
    weights    = weights,
    K          = 175.0,
    T          = T3,
    discount   = discount,
    corr       = np.identity(4),
    paths      = 50000
)


# --------- Calculate Prices & Greeks ---------

results = []
for name, obj in [
    ("BHP Euro Call", bhp),
    ("CBA Am Put",    cba),
    ("WES Barrier",   wes)
]:
    price = obj.price()
    delta = obj.delta()
    theta = obj.theta()
    vega = getattr(obj, "vega", lambda: np.nan)()  # only for classes with vega
    results.append((name, price, delta, theta, vega))

# Basket Call Greeks
basket_price = basket.price()
basket_delta = basket.delta()
basket_theta = basket.theta()
basket_vega  = basket.vega()

results.append(("Basket Call", basket_price, basket_delta, basket_theta, basket_vega))

# Optional: Print results
for r in results:
    print(f"{r[0]}: Price={r[1]:.4f}, Delta={r[2]:.4f}, Theta={r[3]:.4f}, Vega={r[4]:.4f}")


BHP Euro Call: Price=8.0146, Delta=0.7156, Theta=-2.0144, Vega=nan
CBA Am Put: Price=12.3417, Delta=-0.4354, Theta=-4.6117, Vega=nan
WES Barrier: Price=14.2683, Delta=0.7655, Theta=-4.9034, Vega=nan
Basket Call: Price=4.9958, Delta=-280.0013, Theta=13.8726, Vega=132.0568


In [5]:
# ── Cell 5: Results Summary ──────────────────────────────────────────
# Assemble into a DataFrame for clear presentation
import pandas as pd

df = pd.DataFrame(results, columns=["Trade", "Price", "Delta", "Theta", "Vega"]).set_index("Trade")
df


,Price,Delta,Theta,Vega
Trade,,,,
BHP Euro Call,8.014636,0.715599,-2.014358,NaN
CBA Am Put,12.341716,-0.435382,-4.611664,NaN
WES Barrier,14.268326,0.765479,-4.903363,NaN
Basket Call,4.995779,-280.001333,13.872608,132.056844


In [6]:
# ── Cell 7: Narrative Hedge Summary ───────────────────────────────────

from IPython.display import Markdown, display

# Rebuild a small DataFrame from your `results` list
cols = ["Price","Delta","Theta","Vega"]
df_narr = (
    pd.DataFrame(results, columns=["Trade"] + cols)
      .set_index("Trade")
      .round(4)
)

# Compute hedge quantity = –Delta (i.e. short Δ to hedge)
df_narr["HedgeQty"] = (-df_narr["Delta"]).round(2)

def action(qty):
    return "buying" if qty > 0 else "selling"

# Start building the markdown text
text = "## Portfolio Hedge Summary\n\n"

# First three single‐stock/exotic legs
meta = {
    "BHP Euro Call":  ("BHP", "call"),
    "CBA Am Put":     ("CBA", "put"),
    "WES Barrier":    ("WES", "barrier call")
}

for trade, (ticker, ctype) in meta.items():
    row   = df_narr.loc[trade]
    act   = action(row.HedgeQty)
    text += (
        f"- **{trade}**  \n"
        f"  Price = **${row.Price:.4f}**, Δ = {row.Delta:.4f}, Vega = {row.Vega:.4f}, Theta = {row.Theta:.4f}  \n"
        f"  → Hedge by **{act}** {abs(row.HedgeQty):.2f} shares of {ticker} per {ctype}.\n\n"
    )

# Now the basket leg
brow   = df_narr.loc["Basket Call"]
bact   = action(brow.HedgeQty)
text += (
    f"- **Four-Stock Basket Call**  \n"
    f"  Price = **${brow.Price:.4f}**, Δ = {brow.Delta:.4f}, Vega = {brow.Vega:.4f}, Theta = {brow.Theta:.4f}  \n"
    f"  → Hedge basket by trading each underlying:  \n"
)

# Distribute across weights
weights = [0.10, 0.35, 0.15, 0.40]
tickers = ["BHP", "CSL", "WDS", "MQG"]
for w, t in zip(weights, tickers):
    sub_qty = (w * brow.HedgeQty).round(2)
    sub_act = action(sub_qty)
    text += f"    - **{sub_act}** {abs(sub_qty):.2f} shares of {t} (weight {int(w*100)}%)  \n"

display(Markdown(text))



## Portfolio Hedge Summary

- **BHP Euro Call**  
  Price = **$8.0146**, Δ = 0.7156, Vega = nan, Theta = -2.0144  
  → Hedge by **selling** 0.72 shares of BHP per call.

- **CBA Am Put**  
  Price = **$12.3417**, Δ = -0.4354, Vega = nan, Theta = -4.6117  
  → Hedge by **buying** 0.44 shares of CBA per put.

- **WES Barrier**  
  Price = **$14.2683**, Δ = 0.7655, Vega = nan, Theta = -4.9034  
  → Hedge by **selling** 0.77 shares of WES per barrier call.

- **Four-Stock Basket Call**  
  Price = **$4.9958**, Δ = -280.0013, Vega = 132.0568, Theta = 13.8726  
  → Hedge basket by trading each underlying:  
    - **buying** 28.00 shares of BHP (weight 10%)  
    - **buying** 98.00 shares of CSL (weight 35%)  
    - **buying** 42.00 shares of WDS (weight 15%)  
    - **buying** 112.00 shares of MQG (weight 40%)  
